In [12]:
import pandas as pd
import numpy as np
import random
import os
import random
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
import shutil
import time
import pandas as pd
import random

# data split(train+test)

In [13]:
random.seed(99)

def moveFile(train_img_Dir):
        img_pathDir = os.listdir(train_img_Dir)                    # Extract the original path of the image
        filenumber = len(img_pathDir)
        test_rate = 0.2                                            # 0.2，means 20% of whole dataset is test
        test_picknumber = int(filenumber*test_rate)                # Take a certain number of pictures from the folder according to the test_rate ratio
        # Select samples to move to test
        sample1 = random.sample(img_pathDir, test_picknumber)      # Randomly select sample images of the number of picknumbers
        print(sample1)
        for i in range(0, len(sample1)):
            sample1[i] = sample1[i][:-5]                           # Remove the extension of the image, this list is required when moving the annotation
        for name in sample1:
            src_img_name1 = train_img_Dir + name
            dst_img_name1 = test_img_Dir + name
            shutil.move(src_img_name1 + ".jpeg", dst_img_name1 + ".jpeg")     # Add the extension of the image to move the image
        return


if __name__ == '__main__':
    # train moves from train
    train_img_Dir = "../Car Damage Images and Index/train/"
    
    # test path: image and annotation directory
    test_img_Dir = "../Car Damage Images and Index/test/"
    if not os.path.exists(test_img_Dir):
        os.makedirs(test_img_Dir)
    
    # Run the Divide Dataset function
    moveFile(train_img_Dir)


['382.jpeg', '339.jpeg', '1366.jpeg', '742.jpeg', '1327.jpeg', '1421.jpeg', '1455.jpeg', '1242.jpeg', '1157.jpeg', '1460.jpeg', '982.jpeg', '343.jpeg', '616.jpeg', '9.jpeg', '929.jpeg', '630.jpeg', '1162.jpeg', '788.jpeg', '54.jpeg', '1365.jpeg', '414.jpeg', '97.jpeg', '768.jpeg', '722.jpeg', '1397.jpeg', '326.jpeg', '352.jpeg', '896.jpeg', '139.jpeg', '1278.jpeg', '489.jpeg', '855.jpeg', '683.jpeg', '261.jpeg', '115.jpeg', '959.jpeg', '611.jpeg', '887.jpeg', '1159.jpeg', '501.jpeg', '22.jpeg', '38.jpeg', '1076.jpeg', '1494.jpeg', '666.jpeg', '1010.jpeg', '420.jpeg', '1373.jpeg', '305.jpeg', '622.jpeg', '1146.jpeg', '49.jpeg', '594.jpeg', '1362.jpeg', '754.jpeg', '490.jpeg', '1092.jpeg', '297.jpeg', '1235.jpeg', '1396.jpeg', '956.jpeg', '1294.jpeg', '451.jpeg', '735.jpeg', '1324.jpeg', '565.jpeg', '1321.jpeg', '1065.jpeg', '12.jpeg', '104.jpeg', '1312.jpeg', '510.jpeg', '899.jpeg', '41.jpeg', '1147.jpeg', '421.jpeg', '802.jpeg', '1156.jpeg', '1260.jpeg', '208.jpeg', '776.jpeg', '1007.j

train 1210
test 302

# 创建data标准存放格式

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
import shutil
import time
import pandas as pd
import random
# set random seed
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [19]:
random.seed(100)
data_dir = '../Car Damage Images and Index/'  
label_file, train_dir, test_dir = 'data.xlsx', 'train', 'test'  
new_data_dir = './train_valid_test'  # The directory where the sorted data is stored
def mkdir_if_not_exist(path):
    # If the directory path does not exist, create the directory
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))
        
def reorg_dog_data(data_dir, label_file, train_dir, test_dir, new_data_dir):
    # Read training data labels
    labels = pd.read_excel(os.path.join(data_dir, label_file))
    #labels = labels.drop(columns='image')
    id2label = {ID: label for ID, label in labels.values}  # (key: value): (id: label)

    # Shuffle the training data randomly
    train_files = os.listdir(os.path.join(data_dir, train_dir))
    random.shuffle(train_files)    

    # train dataset
    train_size = int(len(train_files))  # trainset size
    for i, file in enumerate(train_files):
        img_id = file.split('.')[0]  # file  id.jpg
        img_label = id2label.get(int(img_id))
        if i < train_size:
            mkdir_if_not_exist([new_data_dir, 'train', img_label])
            shutil.copy(os.path.join(data_dir, train_dir, file),
                        os.path.join(new_data_dir, 'train', img_label))

    # test dataset
    test_files = os.listdir(os.path.join(data_dir, test_dir))
    test_size = len(test_files)
    for i, file in enumerate(test_files):
        img_id = file.split('.')[0]
        img_label = id2label.get(int(img_id))
        if i < test_size:
            mkdir_if_not_exist([new_data_dir, 'test', img_label])
            shutil.copy(os.path.join(data_dir, test_dir, file),
                        os.path.join(new_data_dir, 'test', img_label))
random.seed(100)
reorg_dog_data(data_dir, label_file, train_dir, test_dir, new_data_dir)

# image augmentation

In [5]:
random.seed(100)
transform_train = transforms.Compose([
    # Randomly crop the image to an area of 0.08~1 times the original image area, and the ratio of height and width is 3/4~4/3, and then scale it to a new image with height and width of 224 pixels
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0),  
                                 ratio=(3.0/4.0, 4.0/3.0)),
    # Random horizontal flip with probability 0.5
    transforms.RandomHorizontalFlip(),
    # randomly change brightness, contrast, saturation and hue
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.5),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

# Image enhancement on the test set only does deterministic operations
transform_test = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# data loading

In [8]:
new_data_dir = './train_valid_test'
train_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'train'),
                                            transform=transform_train)
test_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'test'),
                                            transform=transform_test)
batch_size = 32   #batch size can change
train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)  # shuffle=False

# k-fold

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.CrossEntropyLoss()

dataset = train_ds
#dataset = ConcatDataset([train_ds, test_ds])

#num_epochs=60
batch_size=32
#k=5
#splits=KFold(n_splits=k,shuffle=True,random_state=42)
#foldperf={}

# CV

for finetuning: param_group=True


for sratch net: param_group=False

In [10]:
def train_batch_ch13(net, X, y, loss, trainer, devices):
    """Train for a minibatch with mutiple GPUs (defined in Chapter 13)."""
    if isinstance(X, list):
        # Required for BERT fine-tuning (to be covered later)
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    net.train()
    trainer.zero_grad()
    pred = net(X)
    l = loss(pred, y)
    l.sum().backward()
    trainer.step()
    train_loss_sum = l.sum()
    train_acc_sum = d2l.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

In [11]:
from d2l import torch as d2l
def train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus()):
    """Train a model with mutiple GPUs (defined in Chapter 13)."""
    timer, num_batches = d2l.Timer(), len(train_iter)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples,
        # no. of predictions
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = train_batch_ch13(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
    
    loss = metric[0] / metric[2]
    train_acc = metric[1] / metric[3]

    return loss,train_acc,test_acc

In [12]:
def cv_result(learning_rate,batch_size,num_epochs,param_group=True):
    loss_ls =[]
    train_acc_ls =[]
    test_acc_ls = []
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

        print('Fold {}'.format(fold + 1))

        #all test there means validation

        train_sampler = SubsetRandomSampler(train_idx) 
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
        
        net = get_net()

        loss,train_acc,test_acc = train_fine_tuning(net, learning_rate, train_loader, test_loader, batch_size, num_epochs,
                          param_group=True)

        loss_ls.append(loss)
        train_acc_ls.append(train_acc)
        test_acc_ls.append(test_acc)
        print('loss: ',loss)
        print('train: ',train_acc)
        print('test: ',test_acc)
    return loss_ls,train_acc_ls,test_acc_ls

In [13]:
def train_fine_tuning(net, learning_rate, trainloader, testloader, batch_size=128, num_epochs=20,
                      param_group=True):
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': learning_rate * 10}],
                                lr=learning_rate, weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.001)
    
    loss,train_acc,test_acc = train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus())
    return loss,train_acc,test_acc